In [1]:
import os
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
# from networkx.algorithms.triads import all_triads, is_triad, triad_type, triadic_census
from utils import read_edgelist, read_network, DATASET_EPINIONS, DATASET_WIKIPEDIA, DATASET_SLASHDOT
from itertools import combinations
import argparse
import pickle

In [2]:
ROOTPATH = os.getcwd()[:-3]

In [3]:
def create_network_weighted(df):
    g = nx.Graph()
    for _, x in df.iterrows():
        if (g.has_edge(x["from"],x["to"])):
            g[x["from"]][x["to"]]["weight"] += 1
        else:
            g.add_edge(x["from"], x["to"], weight=1)
    return g

In [4]:
wiki_df = read_edgelist(DATASET_WIKIPEDIA)
wiki_d = read_network(DATASET_WIKIPEDIA, directed=True)
wiki_w = create_network_weighted(wiki_df)

In [5]:
def search_triangles(g):
    triplets = []
    for n in g.nodes():
        for nb, nb2 in combinations(g[n],2):
            if g.has_edge(nb, nb2):
                triplets.append(frozenset((n, nb, nb2)))
    return list(set(triplets))

In [17]:
wiki_df[(wiki_df["from"]==1)&(wiki_df["to"]==2)]["sign"].values[0]
# df[(df["from"]==1)and(df["to"]==2)]["sign"]

1

In [18]:
def sum_link_sign(srcs, tars, df):
    x = 0
    for s,t in zip(srcs, tars):
        x += df[(df["from"]==s)&(df["to"]==t)]["sign"].values[0]
    return x

In [20]:
def return_triad_type(srcs, tars, df):
    all_t = np.zeros(4)
    if sum_link_sign(srcs, tars, df) == 3:
        all_t[3] += 1
    elif sum_link_sign(srcs, tars, df) == 1:
        all_t[2] += 1
    elif sum_link_sign(srcs, tars, df) == -1:
        all_t[1] += 1
    elif sum_link_sign(srcs, tars, df) == -3:
        all_t[0] += 1
    return all_t

In [7]:
wiki_triads = search_triangles(wiki_w)

In [33]:
def count_triads(triplet, g_directed):
    triplet = list(triplet)
    TRIADS = []
    # 8 possible motifs
    # 1. a->b, a->c, b->c
    if (g_directed.has_edge(triplet[0],triplet[1])) and (g_directed.has_edge(triplet[0],triplet[2])) \
    and (g_directed.has_edge(triplet[1],triplet[2])):
        TRIADS.append(return_triad_type([triplet[0],triplet[0],triplet[1]],
                                        [triplet[1],triplet[2],triplet[2]], wiki_df))
    # 2. a->b, a->c, b<-c
    if (g_directed.has_edge(triplet[0],triplet[1])) and (g_directed.has_edge(triplet[0],triplet[2])) \
    and (g_directed.has_edge(triplet[2],triplet[1])):
        TRIADS.append(return_triad_type([triplet[0],triplet[0],triplet[2]],
                                        [triplet[1],triplet[2],triplet[1]], wiki_df))
    # 3. a->b, a<-c, b->c
    if (g_directed.has_edge(triplet[0],triplet[1])) and (g_directed.has_edge(triplet[2],triplet[0])) \
    and (g_directed.has_edge(triplet[1],triplet[2])):
        TRIADS.append(return_triad_type([triplet[0],triplet[2],triplet[1]],
                                        [triplet[1],triplet[0],triplet[2]], wiki_df))
    # 4. a->b, a<-c, b<-c
    if (g_directed.has_edge(triplet[0],triplet[1])) and (g_directed.has_edge(triplet[2],triplet[0])) \
    and (g_directed.has_edge(triplet[2],triplet[1])):
        TRIADS.append(return_triad_type([triplet[0],triplet[2],triplet[2]],
                                        [triplet[1],triplet[0],triplet[1]], wiki_df))
    # 5. a<-b, a->c, b->c
    if (g_directed.has_edge(triplet[1],triplet[0])) and (g_directed.has_edge(triplet[0],triplet[2])) \
    and (g_directed.has_edge(triplet[1],triplet[2])):
        TRIADS.append(return_triad_type([triplet[1],triplet[0],triplet[1]],
                                        [triplet[0],triplet[2],triplet[2]], wiki_df))
    # 6. a<-b, a->c, b<-c
    if (g_directed.has_edge(triplet[1],triplet[0])) and (g_directed.has_edge(triplet[0],triplet[2])) \
    and (g_directed.has_edge(triplet[2],triplet[1])):
        TRIADS.append(return_triad_type([triplet[1],triplet[0],triplet[2]],
                                        [triplet[0],triplet[2],triplet[1]], wiki_df))
    # 7. a<-b, a<-c, b->c
    if (g_directed.has_edge(triplet[1],triplet[0])) and (g_directed.has_edge(triplet[2],triplet[0])) \
    and (g_directed.has_edge(triplet[1],triplet[2])):
        TRIADS.append(return_triad_type([triplet[1],triplet[2],triplet[1]],
                                        [triplet[0],triplet[0],triplet[2]], wiki_df))
    # 8. a<-b, a<-c, b<-c
    if (g_directed.has_edge(triplet[1],triplet[0])) and (g_directed.has_edge(triplet[2],triplet[0])) \
    and (g_directed.has_edge(triplet[2],triplet[1])):
        TRIADS.append(return_triad_type([triplet[1],triplet[2],triplet[2]],
                                        [triplet[0],triplet[0],triplet[1]], wiki_df))
    return TRIADS

In [37]:
np.sum(count_triads([2466, 5051, 757], wiki_d), axis=0)

array([0., 0., 0., 2.])

In [9]:
triads_dict = defaultdict(list)

In [45]:
ALL_TRIADS = []
for i,t in enumerate(wiki_triads):
    if i%500==0: print("progress:", i/len(wiki_triads))
    ALL_TRIADS.append(np.sum(count_triads(t, wiki_d), axis=0))

progress: 0.0
progress: 0.0008233447888038282
progress: 0.0016466895776076564
progress: 0.0024700343664114848
progress: 0.0032933791552153127
progress: 0.004116723944019141
progress: 0.0049400687328229696
progress: 0.0057634135216267975
progress: 0.0065867583104306255
progress: 0.007410103099234454
progress: 0.008233447888038281
progress: 0.009056792676842111
progress: 0.009880137465645939
progress: 0.010703482254449767
progress: 0.011526827043253595
progress: 0.012350171832057423
progress: 0.013173516620861251
progress: 0.013996861409665079
progress: 0.014820206198468909
progress: 0.015643550987272735
progress: 0.016466895776076563
progress: 0.017290240564880394
progress: 0.018113585353684222
progress: 0.01893693014248805
progress: 0.019760274931291878
progress: 0.020583619720095706
progress: 0.021406964508899534
progress: 0.022230309297703362
progress: 0.02305365408650719
progress: 0.023876998875311018
progress: 0.024700343664114846
progress: 0.025523688452918674
progress: 0.02634703

progress: 0.22559647213224893
progress: 0.22641981692105276
progress: 0.2272431617098566
progress: 0.22806650649866042
progress: 0.22888985128746425
progress: 0.22971319607626808
progress: 0.23053654086507192
progress: 0.23135988565387572
progress: 0.23218323044267955
progress: 0.23300657523148338
progress: 0.2338299200202872
progress: 0.23465326480909104
progress: 0.23547660959789488
progress: 0.2362999543866987
progress: 0.23712329917550254
progress: 0.23794664396430637
progress: 0.23876998875311017
progress: 0.239593333541914
progress: 0.24041667833071784
progress: 0.24124002311952167
progress: 0.2420633679083255
progress: 0.24288671269712933
progress: 0.24371005748593316
progress: 0.244533402274737
progress: 0.2453567470635408
progress: 0.24618009185234463
progress: 0.24700343664114846
progress: 0.2478267814299523
progress: 0.24865012621875612
progress: 0.24947347100755995
progress: 0.25029681579636376
progress: 0.2511201605851676
progress: 0.2519435053739714
progress: 0.2527668501

progress: 0.455309668208517
progress: 0.45613301299732084
progress: 0.45695635778612465
progress: 0.4577797025749285
progress: 0.4586030473637323
progress: 0.45942639215253617
progress: 0.46024973694133997
progress: 0.46107308173014383
progress: 0.46189642651894763
progress: 0.46271977130775144
progress: 0.4635431160965553
progress: 0.4643664608853591
progress: 0.46518980567416296
progress: 0.46601315046296676
progress: 0.4668364952517706
progress: 0.4676598400405744
progress: 0.4684831848293783
progress: 0.4693065296181821
progress: 0.4701298744069859
progress: 0.47095321919578975
progress: 0.47177656398459356
progress: 0.4725999087733974
progress: 0.4734232535622012
progress: 0.4742465983510051
progress: 0.4750699431398089
progress: 0.47589328792861274
progress: 0.47671663271741654
progress: 0.47753997750622035
progress: 0.4783633222950242
progress: 0.479186667083828
progress: 0.48001001187263187
progress: 0.4808333566614357
progress: 0.48165670145023953
progress: 0.48248004623904334

progress: 0.6883162434400004
progress: 0.6891395882288043
progress: 0.689962933017608
progress: 0.6907862778064119
progress: 0.6916096225952157
progress: 0.6924329673840195
progress: 0.6932563121728234
progress: 0.6940796569616272
progress: 0.6949030017504311
progress: 0.6957263465392348
progress: 0.6965496913280387
progress: 0.6973730361168425
progress: 0.6981963809056463
progress: 0.6990197256944501
progress: 0.699843070483254
progress: 0.7006664152720579
progress: 0.7014897600608616
progress: 0.7023131048496655
progress: 0.7031364496384693
progress: 0.7039597944272731
progress: 0.7047831392160769
progress: 0.7056064840048808
progress: 0.7064298287936847
progress: 0.7072531735824884
progress: 0.7080765183712923
progress: 0.7088998631600961
progress: 0.7097232079489
progress: 0.7105465527377037
progress: 0.7113698975265076
progress: 0.7121932423153114
progress: 0.7130165871041152
progress: 0.713839931892919
progress: 0.7146632766817229
progress: 0.7154866214705268
progress: 0.71630996

progress: 0.9221461634602877
progress: 0.9229695082490914
progress: 0.9237928530378953
progress: 0.9246161978266991
progress: 0.9254395426155029
progress: 0.9262628874043067
progress: 0.9270862321931106
progress: 0.9279095769819145
progress: 0.9287329217707182
progress: 0.9295562665595221
progress: 0.9303796113483259
progress: 0.9312029561371297
progress: 0.9320263009259335
progress: 0.9328496457147374
progress: 0.9336729905035412
progress: 0.934496335292345
progress: 0.9353196800811489
progress: 0.9361430248699527
progress: 0.9369663696587566
progress: 0.9377897144475603
progress: 0.9386130592363642
progress: 0.939436404025168
progress: 0.9402597488139718
progress: 0.9410830936027756
progress: 0.9419064383915795
progress: 0.9427297831803834
progress: 0.9435531279691871
progress: 0.944376472757991
progress: 0.9451998175467948
progress: 0.9460231623355986
progress: 0.9468465071244024
progress: 0.9476698519132063
progress: 0.9484931967020102
progress: 0.9493165414908139
progress: 0.95013

In [51]:
for t in ALL_TRIADS:
    if np.sum(t)==0:
        print("1")

In [54]:
np.sum(ALL_TRIADS, axis=0)

array([  9402.,  67278., 171525., 540842.])

In [49]:
np.sum(np.sum(ALL_TRIADS, axis=0))

789047.0